In [ ]:
# 2. 학습 조교 에이전트 (RAG-수업자료)

from dotenv import load_dotenv
load_dotenv()

import glob, os
from typing import Literal, Optional
from dataclasses import dataclass

# Loaders / Splitters / Vectorstore
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS

# LCEL
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Retrieval enhancers
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

@dataclass
class TAConfig:
    study_dir: str = "./mystudy"
    index_path: Optional[str] = "./faiss_index"   # None이면 메모리 전용
    chunk_size: int = 500
    chunk_overlap: int = 50
    k: int = 5
    model: str = "gpt-5-nano"                     # 원 코드와 호환
    temperature: float = 0.2
    use_multi_query: bool = True
    use_compression: bool = True                 # Embeddings 기반 필터
    similarity_threshold: float = 0.3            # 압축 필터 기준

def load_corpus(study_dir: str):
    pdfs = glob.glob(os.path.join(study_dir, "*.pdf"))
    mds  = glob.glob(os.path.join(study_dir, "*.md"))
    docs = []

    for p in pdfs:
        dlist = PyMuPDFLoader(p).load()
        for d in dlist:
            d.metadata["file_type"] = "pdf"
            d.metadata["source_file"] = os.path.basename(p)
        docs.extend(dlist)

    for p in mds:
        dlist = TextLoader(p, encoding="utf-8").load()
        for d in dlist:
            d.metadata["file_type"] = "markdown"
            d.metadata["source_file"] = os.path.basename(p)
        docs.extend(dlist)

    return docs

def build_vectorstore(docs, cfg: TAConfig):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=cfg.chunk_size, chunk_overlap=cfg.chunk_overlap
    )
    chunks = splitter.split_documents(docs)
    embeddings = OpenAIEmbeddings()

    # 로컬 인덱스가 있으면 재사용
    if cfg.index_path and os.path.isdir(cfg.index_path):
        vs = FAISS.load_local(
            folder_path=cfg.index_path,
            embeddings=embeddings,
            allow_dangerous_deserialization=True,  # 신중 사용
        )
        # 갱신: 신규 청크를 추가(간단 합류)
        vs.add_documents(chunks)
    else:
        vs = FAISS.from_documents(chunks, embeddings)
        if cfg.index_path:
            os.makedirs(cfg.index_path, exist_ok=True)
            vs.save_local(cfg.index_path)

    return vs

def build_retriever(vs: FAISS, cfg: TAConfig):
    base = vs.as_retriever(search_kwargs={"k": cfg.k})

    if cfg.use_multi_query:
        llm_q = ChatOpenAI(model=cfg.model, temperature=0)
        base = MultiQueryRetriever.from_llm(retriever=base, llm=llm_q)

    if cfg.use_compression:
        compressor = EmbeddingsFilter(
            embeddings=OpenAIEmbeddings(),
            similarity_threshold=cfg.similarity_threshold,
        )
        base = ContextualCompressionRetriever(
            base_retriever=base, base_compressor=compressor
        )
    return base

BASE_TEMPLATE = """당신은 제공된 수업자료를 기반으로 답변하는 학습 조교입니다.
규칙:
- 제공된 context 범위 안에서만 답하세요.
- 핵심을 요약하고, 근거가 된 소스 파일명(가능 시 페이지/섹션)을 괄호로 덧붙이세요.
- 자료에 없으면 "제공된 자료에서 해당 정보를 찾을 수 없습니다"라고만 답하세요.

[context]
{context}

[question]
{question}

[answer]
"""

QUIZ_TEMPLATE = """당신은 수업자료 기반 퀴즈 제작 조교입니다.
규칙:
- 5문항 객관식(보기 4개, 정답 표시)으로 생성하세요.
- 각 문항 아래 근거가 된 소스 파일명과 문맥을 간단히 덧붙이세요.
- 자료에 없으면 "제공된 자료에서 퀴즈 근거를 찾을 수 없습니다"라고만 답하세요.

[context]
{context}

[topic_or_question]
{question}

[quiz]
"""

SUMMARY_TEMPLATE = """당신은 수업자료 기반 요약 조교입니다.
규칙:
- 핵심 개념을 5줄 내외로 요약하고, 중요한 용어는 **굵게** 표시하세요.
- 근거가 된 소스 파일명과 문맥을 괄호로 덧붙이세요.
- 자료에 없으면 "제공된 자료에서 해당 내용을 찾을 수 없습니다"라고만 답하세요.

[context]
{context}

[topic]
{question}

[summary]
"""

def build_chain(retriever, template: str, cfg: TAConfig):
    prompt = ChatPromptTemplate.from_template(template)
    llm = ChatOpenAI(model=cfg.model, temperature=cfg.temperature)
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return chain

Task = Literal["qa", "quiz", "summary"]

class StudyTeachingAssistant:
    def __init__(self, cfg: TAConfig = TAConfig()):
        self.cfg = cfg
        docs = load_corpus(cfg.study_dir)
        self.vectorstore = build_vectorstore(docs, cfg)
        self.retriever = build_retriever(self.vectorstore, cfg)
        self.qa_chain = build_chain(self.retriever, BASE_TEMPLATE, cfg)
        self.quiz_chain = build_chain(self.retriever, QUIZ_TEMPLATE, cfg)
        self.summary_chain = build_chain(self.retriever, SUMMARY_TEMPLATE, cfg)

    def invoke(self, question: str, task: Task = "qa") -> str:
        if task == "qa":
            return self.qa_chain.invoke(question)
        if task == "quiz":
            return self.quiz_chain.invoke(question)
        if task == "summary":
            return self.summary_chain.invoke(question)
        raise ValueError("Unsupported task")

    def refresh(self):
        # 신규 파일 반영(간단 재색인+병합)
        docs = load_corpus(self.cfg.study_dir)
        self.vectorstore = build_vectorstore(docs, self.cfg)
        self.retriever = build_retriever(self.vectorstore, self.cfg)
        self.qa_chain = build_chain(self.retriever, BASE_TEMPLATE, self.cfg)
        self.quiz_chain = build_chain(self.retriever, QUIZ_TEMPLATE, self.cfg)
        self.summary_chain = build_chain(self.retriever, SUMMARY_TEMPLATE, self.cfg)

if __name__ == "__main__":
    cfg = TAConfig(
        study_dir="./mystudy",
        index_path="./faiss_index",
        use_multi_query=True,
        use_compression=True,
        k=5,
        model="gpt-5-nano",
    )
    ta = StudyTeachingAssistant(cfg)
    print("=== QA ===")
    print(ta.invoke("강의에서 다룬 RAG의 핵심 구성요소를 정리해줘", task="qa"))
    print("\n=== QUIZ ===")
    print(ta.invoke("지도학습과 비지도학습 차이", task="quiz"))
    print("\n=== SUMMARY ===")
    print(ta.invoke("벡터 임베딩과 유사도 검색", task="summary"))


=== QA ===
- 외부 지식 소스 연결 및 벡터 인덱스: RAG의 기본은 외부 데이터 소스(문서/데이터베이스)를 검색 가능한 상태로 연결하고, 문서 조각을 벡터 공간에 인덱싱해 유사도 기반으로 관련 자료를 찾는 것(랭체인 RAG의 벡터 기반 검색 아이디어 및 외부 소스 연결 접근; 3. 그래프 RAG 특징). 근거: (랭체인 RAG 와 graph RAG의 차이.md, 1. 이번 주에 배운 내용이 랭체인과 RAG를 다루는 데 주는 도움), (랭체인 RAG 와 graph RAG의 차이.md, 3. 그래프 RAG(Graph RAG) 특징).

- 검색/증강 모듈 (Retriever): 질의 맥락에 맞는 문서를 찾아 LLM에 전달하는 역할로, “유사한 것”을 벡터 공간에서 찾아주는 방식이 핵심이라는 설명이 포함됨. 근거: (랭체인 RAG 와 graph RAG의 차이.md, 3. 그래프 RAG 특징), (20250703 perplexity 학습내용.md, 핵심 요약).

- LLM 기반 생성 엔진: 검색된 자료를 바탕으로 최종 응답을 생성하는 생성기 역할이 core로 언급됨. 근거: (20250703 perplexity 학습내용.md, 핵심 요약).

- 컨텍스트 결합 및 증거 제시 기능: 검색 결과를 실제 응답의 증거로 인용하고, 맥락과 함께 제시하는 방식으로 응답의 신뢰성을 높임. 이는 RAG의 “생성에 증거를 연결하는” 특성과도 맞물림. 근거: (20250703 perplexity 학습내용.md, 핵심 요약).

- 그라운딩 및 다중 홉 추론(특히 Graph RAG에서의 차별점): Graph RAG의 특징으로, 지식 그래프를 이용한 다중 홉 추론, 설명 가능성, 데이터 통합 등 복잡한 맥락의 연쇄 추론이 가능한 점이 강조됨. 이 부분은 랭체인 RAG와의 차이를 설명하는 문맥에서도 확인 가능. 근거: (랭체인 RAG 와 graph RAG의 차이.md, 3. 그래프 RAG(Graph RAG) 특징), (랭체인 RAG 와 graph RAG의 차이.md, 4. 요약